In [6]:
import json
import numpy as np
import pandas as pd
import pickle
from scipy.stats import binom, norm

In [11]:
with open('seasons.pickle') as f:
    seasons = pickle.load(f)
for year, season in seasons.iteritems():
    print year,
    season.load_scores()

2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018


In [136]:
scores = pd.read_csv('pd_data/judge_scores.csv')

In [141]:
scores['same_country'] = scores.apply(lambda row: 1 if row.country == row.judge_country else 0, axis=1)

In [142]:
scores['is_larger'] = scores.apply(lambda row: 1 if row.judge_score > row.med_score else 0, axis=1)

In [143]:
scores['dist_med'] = scores.apply(lambda row: row.judge_score - row.med_score, axis=1)

In [144]:
scores_not_med = scores[scores.judge_score != scores.med_score]
scores_med     = scores[scores.judge_score == scores.med_score]

In [145]:
def critical_values(n, cv=1.96):
    lower = -cv * np.sqrt(n*0.25) + 0.5*n + 0.5
    upper =  cv * np.sqrt(n*0.25) + 0.5*n - 0.5
    return (lower, upper)

In [146]:
scores_not_med['judge_n'] = scores_not_med.apply(lambda row: 1 if row.country == row.judge_country else -1, axis=1)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [147]:
scores_not_med['u'] = scores_not_med.apply(lambda row: 1 if row.is_larger == row.same_country else 0, axis=1)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [160]:
len(scores_not_med.judge.unique())

244

In [170]:
nation_count = {}

In [180]:
significant_judges = set()

In [183]:
len(significant_judges), len(scores_not_med.judge.unique())

(104, 244)

In [181]:
for judge in scores_not_med.judge.unique():
    p = test_bias(scores_not_med[scores_not_med.judge == judge])
    if p <= 0.05/244:
        if nations[judge] not in nation_count:
            nation_count[nations[judge]] = 1
        else:
            nation_count[nations[judge]] += 1
        print judge, nations[judge], '*'
        significant_judges.add(judge)
    else:
        print judge, nations[judge]
    print

1.11022302463e-16 0.769616026711 461 599
Cynthia BENSON CAN *

0.999999999997 0.282258064516 70 248
Franco BENINI ITA

1.11022302463e-16 0.740776699029 763 1030
Tianyi ZHANG CHN *

0.00117111841519 0.542426645519 684 1261
Masako KUBOTA JPN

0.00207532339854 0.545086119554 538 987
Alexei BELETSKI ISR

0.999999997482 0.428141912207 712 1663
Yuri GUSKOV KAZ

0.0957479675601 0.55652173913 64 115
Kristina LUNDGREN USA

1.11022302463e-16 0.725609756098 476 656
Lisa JELINEK AUS *

1.11022302463e-16 0.644903397735 968 1501
Maira ABASOVA RUS *

1.55431223448e-15 0.712121212121 235 330
Robert ROSENBLUTH USA *

1.88490112407e-10 0.633644859813 339 535
Mayumi KATO JPN *

1.01030295241e-14 0.642857142857 450 700
Sviatoslav BABENKO RUS *

0.000138639812114 0.547819971871 779 1422
Tanay OZKAN TUR *

1.11022302463e-16 0.621473354232 793 1276
Chihee RHEE KOR *

0.95737936475 0.480519480519 962 2002
Agita ABELE LAT

0.00637960593868 0.53732718894 583 1085
Anthony LEROY FRA

3.93934496077e-05 0.600536193

In [178]:
#from collections import Counter
# counter = Counter([nation for judge, nation in nations.iteritems() if judge in scores_not_med.judge.unique()])
# for nation in counter:
#     print nation, nation_count.get(nation), counter[nation]

BEL 1 1
ISR 2 4
NED 1 2
SWE 1 5
AZE None 1
SUI 1 8
BGR None 1
GBR 4 6
CAN 13 18
DEN 1 2
KOR 3 6
CRO 1 1
TPE None 1
GER 9 10
USA 13 19
HUN 1 6
UZB None 2
POL 2 5
BUL None 1
GEO None 1
HKG 1 1
FRA 3 10
EST None 2
SVK 3 6
LTU None 1
ESP 1 4
TWN None 1
CHN 6 11
UKR 1 7
ROM None 1
AUS 4 12
SLO None 3
AUT None 6
KAZ None 4
BLR 2 3
FIN 2 6
NZL None 2
JPN 7 18
ZAF 1 1
TUR 1 5
ITA 8 13
LAT 1 3
RUS 6 10
MEX None 2
NOR 1 4
CZE 3 8


In [177]:
nation_count, Counter([nation for judge, nation in nations.iteritems() if judge in scores_not_med.judge.unique()])

({u'AUS': 4,
  u'BEL': 1,
  u'BLR': 2,
  u'CAN': 13,
  u'CHN': 6,
  u'CRO': 1,
  u'CZE': 3,
  u'DEN': 1,
  u'ESP': 1,
  u'FIN': 2,
  u'FRA': 3,
  u'GBR': 4,
  u'GER': 9,
  u'HKG': 1,
  u'HUN': 1,
  u'ISR': 2,
  u'ITA': 8,
  u'JPN': 7,
  u'KOR': 3,
  u'LAT': 1,
  u'NED': 1,
  u'NOR': 1,
  u'POL': 2,
  u'RUS': 6,
  u'SUI': 1,
  u'SVK': 3,
  u'SWE': 1,
  u'TUR': 1,
  u'UKR': 1,
  u'USA': 13,
  u'ZAF': 1},
 Counter({u'AUS': 12,
          u'AUT': 6,
          u'AZE': 1,
          u'BEL': 1,
          u'BGR': 1,
          u'BLR': 3,
          u'BUL': 1,
          u'CAN': 18,
          u'CHN': 11,
          u'CRO': 1,
          u'CZE': 8,
          u'DEN': 2,
          u'ESP': 4,
          u'EST': 2,
          u'FIN': 6,
          u'FRA': 10,
          u'GBR': 6,
          u'GEO': 1,
          u'GER': 10,
          u'HKG': 1,
          u'HUN': 6,
          u'ISR': 4,
          u'ITA': 13,
          u'JPN': 18,
          u'KAZ': 4,
          u'KOR': 6,
          u'LAT': 3,
          u'LTU': 1,

In [148]:
owg_scores_not_med = scores_not_med[scores_not_med.segment_name.str.startswith('owg2018')]

In [223]:
len(owg_scores_not_med), len(scores_med[scores_med.segment_name.str.startswith('owg2018')])

(10078, 13457)

In [210]:
def test_bias(df):
    s = sum(df.u)
    n = len(df)
    print (1-binom.cdf(s, n, 0.5)), '%.3f %d %d' % (float(s) / n, s, n)
    return 1-binom.cdf(s, n, 0.5)

In [194]:
owg_scores_not_med.columns

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'all_scores', u'country',
       u'discipline', u'elt_bonus', u'elt_bv', u'elt_info', u'elt_name',
       u'elt_number', u'elt_points', u'elt_score', u'is_comp', u'judge',
       u'judge_num', u'judge_score', u'med_score', u'num_entries',
       u'overall_rank', u'season', u'segment_name', u'segment_rank', u'skater',
       u'start_order', u'trimmed_mean', u'judge_country', u'same_country',
       u'is_larger', u'dist_med', u'judge_n', u'u'],
      dtype='object')

In [221]:
for discipline in ('men', 'ladies', 'pairs', 'dance'):
    print discipline,
    df = owg_scores_not_med[owg_scores_not_med.segment_name.str.contains(discipline)]
    test_bias(df)
    df_top6 = df[df.segment_rank <= 6]
    print ' top6',
    test_bias(df_top6)
    df_bottom = df[df.segment_rank > df.num_entries * 2/3]
    print ' bottom',
    test_bias(df_bottom)
    df_elt = df[df.is_comp == 0]
    print ' elt',
    test_bias(df_elt)
    df_comp = df[df.is_comp == 1]
    print ' comp',
    test_bias(df_comp)
    for segment in ('short', 'free'):
        print segment,
        df = owg_scores_not_med[owg_scores_not_med.segment_name.str.contains(discipline + '_' + segment)]
        test_bias(df)
        df_top6 = df[df.segment_rank <= 6]
        print 'top6',
        test_bias(df_top6)
        df_bottom = df[df.segment_rank > df.num_entries * 2/3]
        print 'bottom',
        test_bias(df_bottom)
        
        print ' elt',
        df_elt = df[df.is_comp == 0]
        test_bias(df_elt)
        df_elt_top6 = df_top6[df_top6.is_comp == 0]
        print '  top6',
        test_bias(df_elt_top6)
        df_elt_bottom = df_bottom[df_bottom.is_comp == 0]
        print '  bottom',
        test_bias(df_elt_bottom)
        
        print ' comp',
        df_comp = df[df.is_comp == 1]
        test_bias(df_comp)
        df_comp_top6 = df_top6[df_top6.is_comp == 1]
        print '  top6',
        test_bias(df_comp_top6)
        df_comp_bottom = df_bottom[df_bottom.is_comp == 1]
        print '  bottom',
        test_bias(df_comp_bottom)
    print

men 1.61546491739e-08 0.550 1645 2989
 top6 1.30417898703e-12 0.632 437 692
 bottom 0.133253895541 0.520 379 729
 elt 0.00059304840887 0.542 806 1488
 comp 2.11687584728e-06 0.559 839 1501
short 0.170347335209 0.512 772 1508
top6 7.21421617278e-06 0.623 188 302
bottom 0.570904711993 0.495 248 501
 elt 0.531625746865 0.498 316 635
  top6 0.000833756942951 0.629 88 140
  bottom 0.444476172415 0.502 103 205
 comp 0.0878837578065 0.522 456 873
  top6 0.00104326174383 0.617 100 162
  bottom 0.614299872637 0.490 145 296
free 2.09809947194e-12 0.589 873 1481
top6 1.38647370251e-08 0.638 249 390
bottom 0.0101254897756 0.575 131 228
 elt 5.65461029534e-06 0.574 490 853
  top6 4.00561918568e-06 0.640 155 242
  bottom 0.0506838267665 0.570 69 121
 comp 1.28095489771e-08 0.610 383 628
  top6 0.000348957193755 0.635 94 148
  bottom 0.040676524103 0.579 62 107

ladies 2.51032528098e-12 0.565 1583 2802
 top6 1.81696004531e-06 0.592 367 620
 bottom 0.119976291301 0.522 363 696
 elt 4.16261174319e-06 0

In [152]:
test_bias(ladies_short[ladies_short.skater == 'Evgenia MEDVEDEVA'])

2.8519058187e-05 0.764705882353 39 51


In [153]:
ladies_short.columns

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'all_scores', u'country',
       u'discipline', u'elt_bonus', u'elt_bv', u'elt_info', u'elt_name',
       u'elt_number', u'elt_points', u'elt_score', u'is_comp', u'judge',
       u'judge_num', u'judge_score', u'med_score', u'num_entries',
       u'overall_rank', u'season', u'segment_name', u'segment_rank', u'skater',
       u'start_order', u'trimmed_mean', u'judge_country', u'same_country',
       u'is_larger', u'dist_med', u'judge_n', u'u'],
      dtype='object')

In [134]:
test_bias(ladies_short[ladies_short.skater == 'Evgenia MEDVEDEVA'])

0.0120464538492 0.647058823529 33 51


In [58]:
test_bias(ladies_short[ladies_short.skater == 'Kaetlyn OSMOND'])

0.063173537907 0.596153846154 31 52


In [52]:
s = sum(owg_scores_not_med.u)
n = len(owg_scores_not_med)
print 1-binom.cdf(s, n, 0.5), float(s) / n, s, n

1.11022302463e-16 0.56588608851 5703 10078


In [48]:
sum(scores_not_med.country_high_low), sum(scores_not_med.u)

(66809, 66809)

In [88]:
len(scores_not_med), len(scores), len(scores_med)

(108447, 247278, 138831)

In [93]:
n = len(scores_not_med)
1-binom.cdf(61106, n, 0.5), 61106./108447

(1.1102230246251565e-16, 0.5634641806596771)

In [101]:
relscores = scores_not_med[scores_not_med.segment_rank]
s = sum(relscores.country_high_low)
n = len(relscores)
1-binom.cdf(s, n, 0.5), float(s)/n, s, n

(0.00025132609080258383, 0.5175411680474583, 5060, 9777)

In [90]:
for description, is_comp, segment_name in (('short elts', 0, 'short'), ('short comp', 1, 'short'),
                                           ('free elts', 0, 'free'), ('free comp', 1, 'free')):
    print description
    relscores = scores_not_med[scores_not_med.is_comp == is_comp]
    relscores = relscores[relscores.segment_name.str.contains(segment_name)]
    N = len(relscores)
    S = sum(relscores.country_high_low)
    print S, N, '%.20f' % (1 - binom.cdf(S, N, 0.5)), float(S) / N
    relscores = relscores[relscores.same_country == 1]
    N = len(relscores)
    S = sum(relscores.country_high_low)
    print S, N, '%.20f' % (1 - binom.cdf(S, N, 0.5)), float(S) / N

 short elts
11069 19841 0.00000000000000011102 0.557885187239
1321 1672 0.00000000000000011102 0.790071770335
short comp
17744 31391 0.00000000000000011102 0.565257557899
2367 2772 0.00000000000000011102 0.853896103896
free elts
17140 30704 0.00000000000000011102 0.558233454924
2151 2763 0.00000000000000011102 0.778501628664
free comp
15153 26511 0.00000000000000011102 0.571574063596
2147 2532 0.00000000000000011102 0.84794628752


In [73]:
scores_not_med.discipline.unique()

array(['men', 'ladies', 'pairs', 'ice_dance'], dtype=object)

In [103]:
for discipline in scores_not_med.discipline.unique():
    print discipline
    relscores = scores_not_med[scores_not_med.discipline == discipline]
#     relscores = relscores[relscores.same_country == 1]
    N = len(relscores)
    S = sum(relscores.country_high_low)
    print S, N, '%.20f' % (1-binom.cdf(S, N, 0.5)), float(S) / N
    relscores = relscores[relscores.segment_rank <= 6]
    N = len(relscores)
    S = sum(relscores.country_high_low)
    print S, N, '%.20f' % binom.cdf(S, N, 0.5), float(S) / N

men
17958 32992 0.00000000000000011102 0.544313773036
7489 13323 0.99999999999999988898 0.562110635743
ladies
17497 31510 0.00000000000000011102 0.555284036814
7472 13071 0.99999999999999988898 0.57164715783
pairs
11794 20807 0.00000000000000011102 0.566828471188
7729 13200 0.99999999999999988898 0.58553030303
ice_dance
13857 23138 0.00000000000000011102 0.598884951163
6673 10697 0.99999999999999988898 0.62381976255


In [42]:
critical_values(n), n * 0.5

((53901.273337327082, 54545.726662672918), 54223.5)

In [39]:
len(scores_not_med), len(scores_med)

(108447, 138831)

In [173]:
scores_low = {}
scores_high = {}
for judge in elts_rus.judge.unique():
    elts = elts_rus[elts_rus.judge == judge]
    comp = comp_rus[comp_rus.judge == judge]
    n = len(elts) + len(comp)
    lower, upper = critical_values(n, 3.646383)
    obs = sum(elts.is_larger) + sum(comp.is_larger)
    if obs <= lower:
        scores_low[judge] = norm.cdf(obs, loc=n*0.5, scale=np.sqrt(n*0.25))
    elif obs >= upper:
        scores_high[judge] = 1 - norm.cdf(obs, loc=n*0.5, scale=np.sqrt(n*0.25))

In [174]:
len(elts_rus.judge.unique()), len(scores_low), len(scores_high)

(188, 30, 26)

In [168]:
len(elts_rus.judge.unique()), len(scores_low), len(scores_high)

(220, 48, 34)

In [193]:
nations['Virpi KUNNAS-HELMINEN'] = 'FIN'
nations['Vessela POPOVA'] = 'BGR'
nations['Yuko OGAWA'] = 'JPN'
nations['Kvetoslava MATEJOVA'] = 'SVK'
nations['Vladimir CUCHRAN'] = 'SVK'
nations['Thomas BIEGLER'] = 'AUT'
nations['Yoko KUNO'] = 'JPN'
nations['Rebecca ANDREW'] = 'AUS'
nations['Hege ROSTO JENSEN'] = 'NOR'
nations['Igor PROKOP'] = 'SVK'
nations['Senem AHISKAL'] = 'TUR'
nations['Margaret WORSFOLD'] = 'GBR'
nations['Ann FINDLAY'] = 'GBR'
nations['Elisabeth BINDER'] = 'AUT'
nations['Elizabeth ALEXANDRE'] = 'AUS'
nations['Tetsuo ABE'] = 'JPN'
nations['Deborah NOYES'] = 'AUS'
nations['Magdalena RUSIECKA'] = 'POL'
nations['Hanna THEN'] = 'POL'

In [195]:
nations['Simona SPALLUTO'] = 'ITA'
nations['Agnieszka SWIDERSKA'] = 'POL'
nations['Osman SIRVAN'] = 'TUR'
nations['Ariadna MORONES NEGRETE'] = 'MEX'
nations['Susan PETRICEVIC'] = 'NZL'
nations['Mona ADOLFSEN'] = 'NOR'
nations['Darja GABROVSEK-POLAJNAR'] = 'SLO'
nations['Nadezhda FIODOROVA'] = 'BLR'
nations['Alison RYAN'] = 'AUS'
nations['Donatella LEONELLI'] = 'SUI'
nations['Adrienn SCHADENBAUER'] = 'AUT'
nations['Neil GARRARD'] = 'ZAF'
nations['Ernestien BAKKER'] = 'NED'
nations['Joanna MILLER'] = 'AUS'
nations['Ebru ANILDI'] = 'TUR'
nations['Edith SCHILLER'] = 'AUT'
nations['Ekaterina SEROVA'] = 'BLR'
nations['Ko-Man TING'] = 'TWN'
nations['Jitka MOKRA'] = 'CZE'
nations['Etsuko AZUMA'] = 'JPN'

In [197]:
nations['Tae Ri LEE'] = 'KOR'
nations['Akiko KOBAYASHI'] = 'JPN'
nations['Ece ESEN'] = 'TUR'
nations['Kersten BELLMANN'] = 'GER'
nations['Roberts KRUTKRAMELIS'] = 'LAT'
nations['Mary CHAPMAN'] = 'GBR'
nations['Josip CEROVAC'] = 'CRO'

In [199]:
with open('pd_data/judge_nations.json', 'w') as f:
    json.dump(nations, f)

In [109]:
sum(elts_not_med[elts_not_med.judge == 'Weiguang CHEN'].is_larger)

279

In [110]:
len(elts_not_med[elts_not_med.judge == 'Weiguang CHEN'])

629

In [70]:
panels = {}  # map segment name to judging panel
judge_names = set()
for season in seasons.values():
    for event in season.events:
        for discipline in event.disciplines:
            for segment in discipline.segments:
                if 'Function' in segment.panel.officials:
                    del segment.panel.officials['Function']
                    print event, segment
                panel = {}
                for function, official in segment.panel.officials.iteritems():
                    name = remove_mr_ms(official.name)
                    name = all_name_fixes.get(name, name)
                    panel[function] = name
                    if len(name) < 5:
                        print event, segment, name
                    if 'Judge' in official.function and season.champ_year in (2017, 2018):
                        judge_names.add(name)
                panels[event.name + '_' + segment.name] = panel

gpusa2016 gpusa2016 pairs_short -


In [71]:
judge_names.remove('-')

In [72]:
judges = {k: ','.join([v['Judge No.{0}'.format(judge_num+1)] for judge_num in xrange(len(v)-6)])
          for k, v in panels.iteritems()}

In [73]:
elts_df['judges'] = elts_df.apply(lambda row: judges[row.segment_name], axis=1)
comp_df['judges'] = comp_df.apply(lambda row: judges[row.segment_name], axis=1)

In [74]:
elts_df['elt_goes'] = elts_df.apply(lambda row: row.elt_goes if row.segment_name != 'gpusa2016_pairs_short'
              else ','.join([goe for goe in row.elt_goes.split(',') if goe != '-']), axis=1)

In [75]:
comp_df['comp_scores'] = comp_df.apply(lambda row: row.comp_scores if row.segment_name != 'gpusa2016_pairs_short'
              else ','.join([goe for goe in row.comp_scores.split(',') if goe != '-']), axis=1)

In [76]:
elts_df['median_goe'] = elts_df.apply(lambda row: np.median(map(float, row.elt_goes.split(','))), axis=1)

In [77]:
comp_df['median_score'] = comp_df.apply(lambda row: np.median(map(float, row.comp_scores.split(','))), axis=1)

In [78]:
elts_df

,country,discipline,elt_bonus,elt_bv,elt_finalgoe,elt_goes,elt_info,elt_name,elt_number,elt_points,season,segment_name,segment_rank,skater,start_order,judges,median_goe
0,JPN,men,False,12.30,-0.51,"-1.0,0.0,0.0,-1.0,-1.0,0.0,1.0,-1.0,0.0",,4F,1,11.79,2017,gpusa2016_men_short,1,Shoma UNO,9,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran...",0.0
1,JPN,men,False,13.30,-4.00,"-3.0,-3.0,-3.0,-3.0,-3.0,-2.0,-3.0,-3.0,-3.0",<,4T+3T<,2,9.30,2017,gpusa2016_men_short,1,Shoma UNO,9,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran...",-3.0
2,JPN,men,False,3.20,0.86,"1.0,2.0,2.0,2.0,0.0,2.0,2.0,2.0,1.0",,FCSp4,3,4.06,2017,gpusa2016_men_short,1,Shoma UNO,9,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran...",2.0
3,JPN,men,False,3.00,0.64,"1.0,1.0,1.0,1.0,0.0,2.0,1.0,2.0,2.0",,CSSp4,4,3.64,2017,gpusa2016_men_short,1,Shoma UNO,9,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran...",1.0
4,JPN,men,True,9.35,-0.14,"1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0",,3A,5,9.21,2017,gpusa2016_men_short,1,Shoma UNO,9,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran...",0.0
5,JPN,men,False,3.30,0.93,"2.0,1.0,2.0,2.0,1.0,3.0,2.0,2.0,2.0",,StSq3,6,4.23,2017,gpusa2016_men_short,1,Shoma UNO,9,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran...",2.0
6,JPN,men,False,3.50,1.14,"2.0,1.0,3.0,3.0,1.0,2.0,2.0,3.0,3.0",,CCoSp4,7,4.64,2017,gpusa2016_men_short,1,Shoma UNO,9,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran...",2.0
7,USA,men,False,9.60,0.90,"2.0,1.0,1.0,1.0,0.0,1.0,2.0,1.0,2.0",,3F+3T,1,10.50,2017,gpusa2016_men_short,2,Adam RIPPON,10,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran...",1.0
8,USA,men,False,8.50,1.29,"1.0,1.0,2.0,2.0,0.0,1.0,1.0,1.0,2.0",,3A,2,9.79,2017,gpusa2016_men_short,2,Adam RIPPON,10,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran...",1.0
9,USA,men,False,3.50,1.07,"2.0,1.0,3.0,2.0,0.0,3.0,2.0,2.0,3.0",,CCoSp4,3,4.57,2017,gpusa2016_men_short,2,Adam RIPPON,10,"Cynthia BENSON,Lisa JELINEK,Maira ABASOVA,Fran...",2.0


In [103]:
len(elts_df), len(comp_df)

(158173, 89105)

In [120]:
judge_not_med = {}
judge_med = {}
for judge in judge_names:
    obs = len(elts_not_med[elts_not_med.judge == judge])
    obs += len(comp_not_med[comp_not_med.judge == judge])
    judge_not_med[judge] = obs
    
    obs = len(elts_med[elts_med.judge == judge])
    obs += len(comp_med[comp_med.judge == judge])
    judge_med[judge] = obs

In [121]:
sorted(judge_not_med.iteritems(), key=lambda (k, v): v)

[(u'Helene CUCUPHAT', 66),
 (u'Jennifer MAST', 85),
 (u'Francis BETSCH', 86),
 (u'Rossella CECCATTINI', 96),
 (u'Yu WANG', 96),
 (u'Elena LISOVA', 97),
 (u'Szilard TOTH', 98),
 (u'Marketa HORKLOVA', 101),
 (u'Prisca BINZ-MOSER', 101),
 (u'Roberts KRUTKRAMELIS', 101),
 (u'Taffy HOLLIDAY', 110),
 (u'Ece ESEN', 112),
 (u'Rebecca ANDREW', 113),
 (u'Kristina LUNDGREN', 115),
 (u'Evgeni ROKHIN', 121),
 (u'Anja RIST', 123),
 (u'Simona SPALLUTO', 127),
 (u'Ingrid Charlotte WOLTER', 128),
 (u'Garry HOPPE', 131),
 (u'Vladimir CUCHRAN', 136),
 (u'Vladislav PETUKHOV', 137),
 (u'Kaoru TAKINO', 140),
 (u'Elizabeth ALEXANDRE', 140),
 (u'Deborah ISLAM', 146),
 (u'Natalia KRUGLOVA', 148),
 (u'Vessela POPOVA', 149),
 (u'Makoto KANO', 149),
 (u'Tetsuo ABE', 149),
 (u'Tomie FUKUDOME', 155),
 (u'Xixia LIU', 159),
 (u'Darja GABROVSEK-POLAJNAR', 159),
 (u'Aigul KUANISHEVA', 160),
 (u'Guona ZHAO', 160),
 (u'Kvetoslava MATEJOVA', 162),
 (u'Walter ZUCCARO', 162),
 (u'Gloria MORANDI', 163),
 (u'Zuzana PLESNIKOVA